# Reforma a Vapor do Metano --> Reator de Equilíbrio

## Apresentação

Este notebook demonstra a formulação e solução de um problema de reator de equilíbrio a uma temperatura e pressão conhecidas. O notebook usa técnicas de programação funcional, incluindo as funções lambda e map do Python.

<center><img src="https://github.com/amandalemette/ENG1818/blob/6fb679e023faf5918633c3fd921cb7b46d914e29/Imagens/im6.png?raw=true"  width=500 height=320 />

<center><img src="https://github.com/amandalemette/ENG1818/blob/6fb679e023faf5918633c3fd921cb7b46d914e29/Imagens/im5.png?raw=true"  width=700 height=350 />

## Problema Proposto

A reforma a vapor de metano 

$$ CH_4(g) + H_2O(g) \longrightarrow CO(g) + 3\,H_2(g) $$

é uma reação endotérmica que produz hidrogênio. 

A reação normalmente é conduzida em relativamente altas temperaturas e pressões em cima de um catalisador heterogêneo. 

Assumindo que a reação é conduzida com 20% de excesso de água a 20 atm, resolva a extensão de equilíbrio da reação em função da temperatura em operação. 


## Solução

Para uma mistura ideal de gases no equilíbrio, 

$$K_{rxn}(T) = \prod_{c=1}^C \left(y_cP\right)^{\nu_c}$$

ou, depois de aplicar ln,

$$\ln K_{rxn}(T) = \sum_{c=1}^C \nu_c \ln\left(y_cP\right)$$

onde $K_{rxn}(T)$ é a constante de equilíbrio, $y_c$ é a fração molar do componente $c$, $\nu_c$ é o coeficiente estequiométrico do componente $c$, $P$ e $T$ são a pressão e a temperatura respectivamente. 

A estratégia para resolver este problema é usar a equação de Van't Hoff para escrever uma função Python para calcular o lado esquerdo da equação como uma função da temperatura, e os balanços materiais dos componentes para resolver a expressão à direita como função da extensão ou do grau de avanço da reação. Então, dadas a temperatura e a pressão, a equação é resolvida para a extensão de equilíbrio da reação usando um algoritmo de cálculo de raízes.



In [1]:
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

## Dados do Problema

Os dados do problema são descritos nas células a seguir por conveniência. Ajuste esses parâmetros para fazer estudos paramétricos de desempenho do reator.

In [2]:
# Reaction Conditions
P = 20
Tmin = 800
Tmax = 1200

# Reactor Feed Specifications
nIn = dict()
nIn['CH4'] = 1
nIn['H2O'] = 1.2*nIn['CH4']
nIn['CO'] = 0
nIn['H2'] = 0

# List of Components
C = nIn.keys()

## Dados dos componentes

Começamos reunindo a entalpia molar e a energia livre de Gibb das espécies que participam da reação nas condições padrão.


Espécies | $\Delta\hat{G}_f^{\circ}$ <br> [kJ/gmol]| $\Delta\hat{H}_f^{\circ}$ <br> [kJ/gmol] | 
-------------------|------------------|------------------|
Monóxido de Carbono (g) | $-137.27$| $-110.53$ |
Hidrogênio (g)| $0$ | $0$ | 
Metano (g)| $-50.49$ | $-74.52$ | 
Agua (g) | $-228.59$ | $-241.83$ | 



In [3]:
# Enthalpy and Free Energy of Formation
Gf = dict()
Hf = dict()

Gf['CO'] = -137270
Hf['CO'] = -110530

Gf['H2'] = 0
Hf['H2'] = 0

Gf['CH4'] = -50490
Hf['CH4'] = -74520

Gf['H2O'] = -228590
Hf['H2O'] = -241830

## Entalpia e Energia Livre de Gibbs 

A entalpia e a energia livre de Gibb's por mol de avanço de reação é dada por

$$\begin{align*}
\Delta\hat{H}_{rxn}^{\circ} & = \sum_{c=1}^C \nu_c \Delta\hat{H}_{f,c}^{\circ} \\
\Delta\hat{G}_{rxn}^{\circ} & = \sum_{c=1}^C \nu_c \Delta\hat{G}_{f,c}^{\circ} 
\end{align*}$$

onde $\nu_c$ é o coeficiente estequiométrico para espécies  $c$.

In [ ]:
# Reaction Stoichiometry
nu = dict()
nu['CO']  = +1
nu['H2']  = +3
nu['CH4'] = -1
nu['H2O'] = -1

Hr = sum([nu[c]*Hf[c] for c in C])
print("Hr = {:7.3f} kJ/gmol".format(Hr/1000))

Gr = sum([nu[c]*Gf[c] for c in C])
print("Gr = {:7.3f} kJ/gmol".format(Gr/1000))

## Constante de Equilíbrio Utilizando a Equação de van't Hoff

A constante de equilíbrio em função da temperatura pode ser estimada usando a equação de Van't Hoff.

$$ \ln K_r(T) = -\frac{1}{R}\left[\frac{\Delta\hat{G}_{rxn}^{\circ} - 
  \Delta\hat{H}_{rxn}^{\circ}}{298} + \frac{\Delta\hat{H}_{rxn}^{\circ}}{T}\right] $$

A equação acima é implementada em Python como uma função de $T$

In [5]:
R = 8.314 # J/K/gmol
def lnKr(T):
    return -((Gr-Hr)/298 + Hr/T)/R

que pode ser plotada sobre a faixa de temperatura de interesse.

In [ ]:
T = np.linspace(Tmin,Tmax,100)
plt.plot(T,[lnKr(T) for T in T])
plt.xlabel('Temperature [K]')
plt.ylabel('$\ln(K_{r})$')
plt.title('Van\'t Hoff Equation')
plt.grid()

## Balanços Materiais

Os balanços materiais podem ser resolvidos para determinar as vazões molares na saída do reator em função da extensão molar (ou grau de avanço) da reação $\dot{\xi}$.

$$\begin{align*}
\dot{n}_{out,CO} & = \dot{n}_{in,CO} + \nu_{CO}\dot{\xi} \\
\dot{n}_{out,H_2} & = \dot{n}_{in,H_2} + \nu_{H_2}\dot{\xi} \\
\dot{n}_{out,CH_4} & = \dot{n}_{in,CH_4} + \nu_{CH_4}\dot{\xi} \\
\dot{n}_{out,H_2O} & = \dot{n}_{in,H_2O} + \nu_{H_2O}\dot{\xi} 
\end{align*}$$



Os balanços materiais são implementados como funções de extensão molar $x$. As funções são atribuídas a um dicionário nOut. (O argumento padrão extra c=c é adicionado para que o c referenciado na função seja o mesmo que a chave do dicionário. Sem o argumento c=c, o c dentro da função refere-se à variável do contador em vez do desejado valor da chave.)

In [ ]:
C

In [8]:
nOut = dict()
for c in C:
    nOut[c] = lambda x,c=c: nIn[c] + nu[c]*x

Os fluxos molares de saída podem ser plotados como funções da extensão molar (ou grau de avanço) da reação. O primeiro passo é calcular o grau de avanço máximo possível e, em seguida, traçar cada um dos fluxos molares em função do grau de avanço.

In [ ]:
xMax = np.Inf
for c in C:
    if nu[c] < 0:
        xMax = min(xMax,-nIn[c]/nu[c])

print('Maximum molar extent of reaction = {:.2f}'.format(xMax))

x = np.linspace(0,xMax)

for c in C:
    plt.plot(x,[nOut[c](x) for x in x])

plt.legend(C)
plt.xlabel('Molar Extent of Reaction')
plt.ylabel('$n_{Out}$')
plt.title('Molar flows out the Reactor as Function of the Extent of Reaction')
plt.grid()

## Composição dos Gases na saída do Reator

A composição dos gases na corrente de saída é dada por 

$$ y_{n}(\dot{\xi}) = \frac{\dot{n}_{out,n}(\dot{\xi})}{\dot{n}_{Total}(\dot{\xi})} $$

onde o fluxo molar total é a soma dos fluxos molares dos componentes. 

$$\begin{align*}
\dot{n}_{Total} & = \sum_{c=1}^C \dot{n}_{out,c} \\
& = \sum_{c=1}^C \dot{n}_{in,c} + \left(\sum_{c=1}^C \nu_c\right)\dot{\xi}
\end{align*}$$

In [12]:
def nTotal(x):
    nTotal = 0
    for c in C:
        nTotal += nOut[c](x)
    return nTotal

O fluxo molar total é usado para criar um dicionário de funções lambda para a composição do efluente do reator em função do grau de avanço da reação.

In [13]:
y = dict()
for c in C:
    y[c] = lambda x,c=c: nOut[c](x)/nTotal(x)

Plotando a composição como função do grau de avanço da reação.

In [ ]:
x = np.linspace(0,1)
for c in C:
    plt.plot(x,[y[c](x) for x in x])
plt.legend(C)
plt.xlabel('Molar Extent of Reaction')
plt.ylabel('$y_c$')
plt.title('Composition as a Function of Molar Extent of Reaction')
plt.grid()

## Quociente de Reação

Para uma reação na fase gasosa envolvendo uma mistura de gases ideais, definimos o quociente de reação como 

$$K_a(\dot{\xi}) = \prod_{c=1}^C \left(y_c(\dot{\xi})P\right)^{\nu_c}$$

onde $y_c(\dot{\xi})P$ é a pressão parcial do componente $c$. 

Aplicando o logaritmo

$$ \ln K_a(\dot{\xi}) = \sum_{c=1}^C \nu_c\ln(y_c(\dot{\xi})P)$$

In [15]:
def lnKa(x):
    lnKa = 0;
    for c in C:
        lnKa += nu[c]*np.log(P*y[c](x))
    return lnKa

In [ ]:
x = np.linspace(0.001,1)
plt.plot(x,[lnKa(x) for x in x])
plt.xlabel('Molar Extent of Reaction')
plt.ylabel('$\ln(Ka)$')
plt.title('Reaction Activity Quotient');
plt.grid()

## Resolvendo para o grau de avanço da reação no equilíbrio

Para uma dada temperatura e pressão, a solução para o grau de avanço da reação no equilíbrio eé o valor de $\dot{\xi}$ para o qual $K_r(T) = K_a(\dot{\xi})$ que pode ser escrito em termos de logaritmo como

$$\ln K_r(T) = \ln K_a(\dot{\xi})$$

Plotando essas funções lado a lado mostra uma técnica gráfica simples para encontrar soluções para $\dot{\xi}$.

In [ ]:
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(T,[lnKr(T) for T in T])
plt.ylim(-5,10)
plt.xlabel('Temperature [K]')
plt.ylabel('$\ln(Kr)$')
plt.title('Equibrium Constant')
plt.grid()

plt.subplot(1,2,2)
plt.plot(x,[lnKa(x) for x in x])
plt.ylim(-5,10)
plt.xlabel('Molar Extent of Reaction')
plt.ylabel('$ln(Ka)$')
plt.title('Equilibrium Activity Quotient')
plt.grid()

Vamos deixar $\dot{\xi}_{eq}(T)$ denotar os valores de equilíbrio do grau de avanço da reação em função da temperatura. Esses valores são definidos como raízes da equação

$$ \ln K_a(\dot{\xi}_{eq}(T)) - \ln K_r(T) = 0$$

Essa expressão é implementada como uma função lambda do Python, onde um algoritmo root=finding é usado para resolver a condição de equilíbrio em função da temperatura.

In [18]:
from scipy.optimize import brentq as fzero

xEquil = lambda T: fzero(lambda x: lnKa(x) - lnKr(T), 0, xMax)

In [ ]:
plt.plot(T,[xEquil(T) for T in T])
plt.xlabel('Temperature [K]')
plt.ylabel('Extent of Reaction [gmol/min]')
plt.title('Equilibrium Extent of Reaction at {:.1f} atm'.format(P))
plt.grid()

## Composição de Equilíbrio

Agora que temos uma função para calcular o grau de avanço no equilíbrioem função da temperatura, temos tudo o que é necessário para calcular uma variedade de métricas de desempenho para o reator. 

Por exemplo, podemos usar as funções que já criamos para a composição do gás de saída para calcular e plotar a composição do gás de saída do reator em função da temperatura.

In [ ]:
yEquil = dict()
for c in C:
    yEquil[c] = lambda T,c=c: y[c](xEquil(T))

for c in C:
    plt.plot(T,[yEquil[c](T) for T in T])

plt.legend(C)
plt.xlabel('Temperature [K]')
plt.ylabel('Mole Fraction [mol/mol]')
plt.title('Reactor Outlet Gas Composition at {:.1f} atm'.format(P))
plt.grid()


## Conversão do Metano

$$f_{conv} = \frac{\dot{n}_{in,CH_4} - \dot{n}_{out,CH_4}}{\dot{n}_{in,CH_4}}$$

In [ ]:
fconv = lambda T: (nIn['CH4'] - nOut['CH4'](xEquil(T)))/nIn['CH4']

plt.plot(T,[fconv(T) for T in T])
plt.xlabel('Temperature [K]')
plt.ylabel('Fractional Methane Conversion')
plt.title('Equilibrium Conversion at {:.1f} atm'.format(P))
plt.grid()

# Exercícios

1. Repita os cálculos para diferentes pressões operacionais. Como o grau de avanço da reação depende da pressão de operação? Por que este é o caso?
2. Repita os cálculos para diferentes taxas de alimentação para água e metano. Como o grau de avanço da reação depende da razão de alimentação da água?